In [281]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from scipy.spatial.distance import cdist

Put all csv's in a list so they can later be merged.

In [282]:
partial_df_list = [
    pd.read_csv("data/icu_data_28_01.csv"),
    pd.read_csv("data/icu_data_30_02.csv"),
    pd.read_csv("data/icu_data_38_03.csv"),
    pd.read_csv("data/icu_data_43_05.csv"),
    pd.read_csv("data/icu_data_48_05.csv"),
    pd.read_csv("data/icu_data_50_06.csv"),
]

full_df_length = 0

for num, partial_df in enumerate(partial_df_list):
    print(f"The length of dataframe {num} is {len(partial_df)}")
    full_df_length += len(partial_df)

print("The total length of all the dataframes is:", full_df_length)

The length of dataframe 0 is 1746
The length of dataframe 1 is 1746
The length of dataframe 2 is 1746
The length of dataframe 3 is 1746
The length of dataframe 4 is 1746
The length of dataframe 5 is 1746
The total length of all the dataframes is: 10476


Create a dataframe with the partial dataframes.

In [283]:
full_df = pd.concat(partial_df_list)
full_df.head(5)

,Unnamed: 0,distribution,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules
0,0,"[4, 4, 4, 9, 7]",[28],1600,101,93306,68.170254,3272,17,903
1,1,"[4, 4, 4, 10, 6]",[28],1642,175,131442,66.136252,3086,37,952
2,2,"[4, 4, 4, 11, 5]",[28],1679,248,102786,64.277153,3046,62,759
3,3,"[4, 4, 4, 12, 4]",[28],1584,277,98034,59.279599,3144,23,904
4,4,"[4, 4, 4, 13, 3]",[28],1652,372,123672,59.097358,3046,28,908


The first column in the csv is the index column which is not relevant as we do not need to look at specific rows.

In [284]:
full_df.drop(columns="Unnamed: 0", inplace=True)
full_df.head()

,distribution,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules
0,"[4, 4, 4, 9, 7]",[28],1600,101,93306,68.170254,3272,17,903
1,"[4, 4, 4, 10, 6]",[28],1642,175,131442,66.136252,3086,37,952
2,"[4, 4, 4, 11, 5]",[28],1679,248,102786,64.277153,3046,62,759
3,"[4, 4, 4, 12, 4]",[28],1584,277,98034,59.279599,3144,23,904
4,"[4, 4, 4, 13, 3]",[28],1652,372,123672,59.097358,3046,28,908


Let's check if all rows are successfully merged.

In [285]:
print(f"The length of the new df is equal to the total amount of rows in all datasets: {len(full_df) == full_df_length}")

The length of the new df is equal to the total amount of rows in all datasets: True


There are some oddities in the dataset. First of all, the column `bed_amount` contains a list which a single value but 
the column should only contain integers so we should remove the brackets and turn the column into an integer. The 
brackets are removed using `replace()` and the very confusing regular expression `[\[\]]`. The latter consists out of
two parts: the enclosing brackets `[]` and what's within `\[\]`. The enclosing brackets means to select either one of 
the characters between them which we want to be `[` and `]` but they are already regex syntax so we have to escape them
using `\`.

In [286]:
full_df["bed_amount"].replace(r"[\[\]]", "", regex=True, inplace=True)
full_df["bed_amount"] = full_df["bed_amount"].astype(int)

print("The datatype of bed_amount is:", full_df["bed_amount"].dtype)

full_df["bed_amount"].head(5)

The datatype of bed_amount is: int64


0    28
1    28
2    28
3    28
4    28
Name: bed_amount, dtype: int64

Next is the `distribution` column which consists of a list with integers that correspond with the amount of beds 
dedicated to certain specialisms. The order is as following:

In [287]:
specs = ["CARD/INT/OTHER", "NEU/NEC", "CAPU", "CHIR", "COV"]
specs

['CARD/INT/OTHER', 'NEU/NEC', 'CAPU', 'CHIR', 'COV']

We can turn this column into the 5 columns. The column `distribution` is seen as a string so we need to turn it into a
list again. We do that by removing the brackets, splitting the string into a list using `split(",")` and then creating
5 new columns using these lists. After that the columns need to be converted from strings to ints.

In [288]:
full_df["distribution"].replace(r"[\[\]]", "", regex=True, inplace=True)
full_df[specs] = full_df["distribution"].apply(lambda x: x.split(",")).tolist()
full_df[specs] = full_df[specs].astype(int)
full_df.drop(columns="distribution", inplace=True)

full_df.head()

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV
0,28,1600,101,93306,68.170254,3272,17,903,4,4,4,9,7
1,28,1642,175,131442,66.136252,3086,37,952,4,4,4,10,6
2,28,1679,248,102786,64.277153,3046,62,759,4,4,4,11,5
3,28,1584,277,98034,59.279599,3144,23,904,4,4,4,12,4
4,28,1652,372,123672,59.097358,3046,28,908,4,4,4,13,3


We can now save this new dataframe into a csv file.

In [289]:
full_df.to_csv("data/full_monte_carlo.csv")
full_df[(full_df.succesful_reschedules == 208) & (full_df.admissioned == 5465)]

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV
356,50,744,1563,12546,84.413014,5465,2,208,4,10,8,4,24


We do not care about occupancy below `75%` so we can discard those rows.

In [290]:
full_df = full_df[full_df["occupancy"] >= 75]
full_df.head()
print("New length of full_df:", len(full_df))

New length of full_df: 5339


This removes a large amount of rows. Let's focus on hte columns we want to min-max which are `denied`, 
`waiting_time`, `occupancy` and `failed_reschedules`.

In [291]:
min_max_columns = ["denied", "waiting_time", "occupancy", "failed_reschedules"]

Let's standardize these columns and display their distributions.

In [292]:
scaler = StandardScaler()
full_df[min_max_columns] = scaler.fit_transform(full_df[min_max_columns])
full_df.head()

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV
150,28,1.336229,122,0.322829,-1.495968,3387,0.757228,679,4,6,5,6,7
166,28,0.933099,135,-0.155489,-0.735712,3653,2.973738,353,4,6,7,4,7
202,28,2.485748,143,2.086204,-1.233053,3251,-0.312812,934,4,7,4,6,7
203,28,2.737206,157,2.040134,-0.549310,3111,3.279464,955,4,7,4,7,6
210,28,0.805374,139,0.061072,-1.376313,3631,-0.312812,661,4,7,5,5,7


 Let's create a list with the ideal values, which is the 
lowest value for `denied`, `waiting_time` and `failed_reschedules` and the highest value for `occupancy`.

In [293]:
ideal_row = [full_df["denied"].min(), full_df["waiting_time"].min(), full_df["occupancy"].max(), full_df["failed_reschedules"].min()]
print(ideal_row)

[-2.5394080573632527, -1.2368977892934139, 3.5140074359835376, -1.153557235117552]


The ideal row is the theoretical best we can achieve with our dataset. We can now look through all the points and 
calculate the euclidean distance. The row with the smallest distance is the row that gets the closest to our theoretical
distance.

In [294]:
full_df["euclidean_distances"] = cdist(full_df[min_max_columns], np.array([ideal_row]))
full_df.head()

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV,euclidean_distances
150,28,1.336229,122,0.322829,-1.495968,3387,0.757228,679,4,6,5,6,7,6.797372
166,28,0.933099,135,-0.155489,-0.735712,3653,2.973738,353,4,6,7,4,7,6.951433
202,28,2.485748,143,2.086204,-1.233053,3251,-0.312812,934,4,7,4,6,7,7.715998
203,28,2.737206,157,2.040134,-0.549310,3111,3.279464,955,4,7,4,7,6,8.645451
210,28,0.805374,139,0.061072,-1.376313,3631,-0.312812,661,4,7,5,5,7,6.123265


Now we can simply select the row with the lowest euclidean distance. The standardized columns will still be standardized
but this could be fixed by manually looking up the row corresponding to the other values.

In [295]:
full_df[full_df["euclidean_distances"] == full_df["euclidean_distances"].min()]

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV,euclidean_distances
356,50,-1.409846,1563,-0.972522,1.606138,5465,-1.000694,208,4,10,8,4,24,2.23811
